# Associate IAM Policies with EKS Worker Nodes


In [1]:
%%bash

### Source the environment
source ~/.bash_profile

export INSTANCE_ROLE_NAME=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .RoleName")
echo "export INSTANCE_ROLE_NAME=${INSTANCE_ROLE_NAME}" | tee -a ~/.bash_profile

export INSTANCE_PROFILE_ARN=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .Arn")
echo "export INSTANCE_PROFILE_ARN=${INSTANCE_PROFILE_ARN}" | tee -a ~/.bash_profile

export INSTANCE_ROLE_NAME=eksctl-cluster-nodegroup-ng-aaf76-NodeInstanceRole-12ISDU80VNWW1
export INSTANCE_PROFILE_ARN=arn:aws:iam::146478650644:role/eksctl-cluster-nodegroup-ng-aaf76-NodeInstanceRole-12ISDU80VNWW1


# Attach Roles
* Allow Access from/to the Elastic Container Registry (ECR)
* This allows our cluster worker nodes to load custom Docker images (ie. models) from ECR. 
* We will load these custom Docker images in a later section.

In [2]:
%%bash

source ~/.bash_profile

aws iam attach-role-policy --role-name $INSTANCE_ROLE_NAME --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess

echo "Completed"

Completed


# Associated IAM and OIDC
To use IAM Roles for Service Accounts in your cluster, you must create an OIDC identity provider in the IAM console.  

See https://docs.aws.amazon.com/eks/latest/userguide/enable-iam-roles-for-service-accounts.html for more info.

# _This may take 5-10 minutes.  Please be patient._

# _If you see `retryable error`s below, this is OK!_

In [3]:
%%bash

### Source the environment
source ~/.bash_profile

eksctl utils associate-iam-oidc-provider --cluster ${AWS_CLUSTER_NAME} --approve

echo "Completed"

[ℹ]  eksctl version 0.28.1
[ℹ]  using region us-east-1
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 50.788002ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 100.933082ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while awaiting headers)) from ec2metadata/GetToken - will retry after delay of 146.36516ms
[!]  retryable error (RequestError: send request failed
caused by: Put "http://169.254.169.254/latest/api/token": context deadline exceeded (Client.Timeout exceeded while await

# _If you see `retryable error`s ^^ above ^^, this is OK!_

In [4]:
%%bash

### Source the environment
source ~/.bash_profile

aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.identity.oidc.issuer" --output text

echo "Completed"

https://oidc.eks.us-east-1.amazonaws.com/id/2AC0F2258EDD52E6091862B6A03C5BD9
Completed


# Update `~/.kube/config` with our new EKS cluster

In [5]:
%%bash

source ~/.bash_profile

aws eks --region ${AWS_REGION} update-kubeconfig --name ${AWS_CLUSTER_NAME} 

Added new context arn:aws:eks:us-east-1:146478650644:cluster/cluster to /home/ec2-user/.kube/config


# Verify Your EKS Cluster


In [6]:
%%bash

kubectl get ns


NAME              STATUS   AGE
default           Active   63m
kube-node-lease   Active   63m
kube-public       Active   63m
kube-system       Active   63m


In [7]:
%%bash

kubectl get nodes


NAME                             STATUS   ROLES    AGE   VERSION
ip-192-168-33-44.ec2.internal    Ready    <none>   56m   v1.17.9-eks-4c6976
ip-192-168-45-124.ec2.internal   Ready    <none>   56m   v1.17.9-eks-4c6976
ip-192-168-6-199.ec2.internal    Ready    <none>   56m   v1.17.9-eks-4c6976
ip-192-168-9-182.ec2.internal    Ready    <none>   56m   v1.17.9-eks-4c6976


In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();